In [ ]:
# Install Layer and Authentication
!pip install layer -U
import layer
layer.login()

In [7]:
# Project initialization and import Layer decorators
layer.init("my-first-project",pip_packages=['matplotlib','sklearn'])
from layer.decorators import dataset, model, resources, fabric
from layer.decorators.assertions import assert_not_null, assert_true

In [53]:
# Create your first dataset
@dataset("my_first_dataset")
@assert_not_null(["target","feat1"])
@resources(path="./data")
def create_dataset():
  from matplotlib import pyplot as plt
  from sklearn.datasets import make_regression
  import pandas as pd
  
  X_train, Y_train = make_regression(n_samples=100, n_features=3, n_targets=1, noise=0.5)

  features = pd.DataFrame(X_train, columns = ['feat1', 'feat2', 'feat3'])
  target = pd.DataFrame(Y_train, columns = ['target'])
  training_df= pd.concat([features, target], axis=1)

  external_df = pd.read_csv("data/external.csv",names=['feat1','feat2','feat3','target'],header=None)
  final_training_df = training_df.append(external_df)

  training_df[['target']].plot(kind='box', title='Target Quartile Analysis')
  layer.log({"Box plots": plt})
  
  return training_df 

In [ ]:
# Run your function remotely
layer.run([create_dataset])

In [58]:
# Create your first model

# Test function for your model
def model_test_function(predictor):
    return (predictor.coef_[0] > 0 and  predictor.coef_[1] > 0 and predictor.coef_[2] > 0) 

# Model build function
@model("my_first_model")
@fabric("f-medium")
@assert_true(model_test_function)
def create_model():
  from sklearn.linear_model import LinearRegression
  from matplotlib import pyplot as plt
  training_data = layer.get_dataset("my_first_dataset:1.1").to_pandas()
  predictor = LinearRegression(n_jobs=-1)
  predictor.fit(X=training_data.iloc[:,:-1], y=training_data.iloc[:,-1])


  importance = predictor.coef_
  plt.bar(['feat1', 'feat2', 'feat3'], importance)

  layer.log({"Coef_1": importance[0],
             "Coef_2": importance[1],
             "Coef_3": importance[2],
             "Feature Importance Plot": plt
             })

  return predictor

In [ ]:
# Run your function remotely
layer.run([create_model])

In [60]:
# Create a predictions dataset by fetching and using the model
@dataset("predictions")
def generate_predictions():
  from sklearn.datasets import make_regression
  import pandas as pd
  from sklearn.metrics import r2_score
  
  X_test, Y_true = make_regression(n_samples=10, n_features=3, n_targets=1, noise=0.5)

  X_test = pd.DataFrame(X_test, columns = ['feat1', 'feat2', 'feat3'])
  Y_true = pd.DataFrame(Y_true, columns = ['target'])
  
  my_model = layer.get_model("my_first_model:1.1").get_train()
  Y_pred_arr = my_model.predict(X_test)
  Y_pred = pd.DataFrame(Y_pred_arr, columns = ['prediction'])

  r2 = r2_score(Y_true, Y_pred)
  layer.log({"R2 Score": r2})

  predictions_df = pd.concat([X_test, Y_true, Y_pred], axis=1)

  return predictions_df

In [ ]:
# Run your function remotely
layer.run([generate_predictions])